<a href="https://colab.research.google.com/github/barnnita-shrestha/demo-repo/blob/main/docs/examples/building_footprints_usa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Footprint Extraction for the USA

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/building_footprints_usa.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
%pip install geoai-py

## Import libraries

In [2]:
import geoai

## Download sample data

In [4]:
raster_url = (
    "/content/sample_data/output.tif"
)
# vector_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train_buildings.geojson"

In [5]:
# raster_path = geoai.download_file(raster_url)
raster_path = raster_url

In [ ]:
vector_path = geoai.download_file(vector_url)

## Initialize building footprint extraction pretrained model

The pretained model is adapted from the Esri [building footprint extraction](https://www.arcgis.com/home/item.html?id=a6857359a1cd44839781a4f113cd5934) model for the USA. Credits to Esri for the model.

In [6]:
extractor = geoai.BuildingFootprintExtractor()

Model path not specified, downloading from Hugging Face...


building_footprints_usa.pth:   0%|          | 0.00/176M [00:00<?, ?B/s]

Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/f519baf6bb6d3c8b37b4f1f98e8d6a57bbc21f57/building_footprints_usa.pth
Model loaded successfully


## Extract building footprints

### Option 1: Extract building footprints as raster

In [ ]:
mask_path = extractor.save_masks_as_geotiff(
    raster_path=raster_path,
    output_path="output_masks.tif",
    confidence_threshold=0.5,
    mask_threshold=0.5,
)

Processing masks with parameters:
- Confidence threshold: 0.5
- Chip size: (512, 512)
- Mask threshold: 0.5
Dataset initialized with 37 rows and 26 columns of chips
Image dimensions: 9934 x 14042 pixels
Chip size: 512 x 512 pixels
Overlap: 25.0% (stride_x=384, stride_y=384)
Processing raster with 241 batches


 22%|██▏       | 53/241 [29:13<1:40:15, 32.00s/it]

Convert raster to vector

In [ ]:
gdf = extractor.masks_to_vector(
    mask_path=mask_path,
    output_path="output_masks.geojson",
    simplify_tolerance=1.0,
)

### Option 2: Extract building footprints as vector

In [ ]:
output_path = "naip_output_buildings.geojson"
gdf = extractor.process_raster(
    raster_path,
    output_path="output_buildings.geojson",
    batch_size=4,
    confidence_threshold=0.5,
    overlap=0.25,
    nms_iou_threshold=0.5,
    min_object_area=100,
    max_object_area=None,
    mask_threshold=0.5,
    simplify_tolerance=1.0,
)

## Regularize building footprints

In [ ]:
gdf_regularized = extractor.regularize_buildings(
    gdf=gdf,
    min_area=100,
    angle_threshold=15,
    orthogonality_threshold=0.3,
    rectangularity_threshold=0.7,
)

## Visualize building footprints

In [ ]:
gdf.head()

In [ ]:
geoai.view_vector_interactive(
    gdf, column="confidence", layer_name="Building", tiles="Satellite"
)

In [ ]:
geoai.view_vector_interactive(
    gdf, column="confidence", layer_name="Building", tiles=raster_url
)

In [ ]:
geoai.view_vector_interactive(
    gdf_regularized, column="confidence", layer_name="Building", tiles=raster_url
)

In [ ]:
extractor.visualize_results(raster_path, gdf, output_path="naip_output_buildings.png")

In [ ]:
extractor.visualize_results(
    raster_path, gdf_regularized, output_path="naip_output_buildings_regularized.png"
)